#Giai đoạn 1: Data Ingestion & Preprocessing (Xử lý Dữ liệu)



In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import Counter
import os


Dữ liệu gốc

In [2]:
G_raw = nx.read_edgelist("/content/facebook_combined.txt", nodetype=int)

print(f"Số nút:  {G_raw.number_of_nodes()}")
print(f"Số cạnh: {G_raw.number_of_edges()}")
print(f"Đồ thị có hướng: {'Có' if G_raw.is_directed() else 'Không'}")


Số nút:  4039
Số cạnh: 88234
Đồ thị có hướng: Không


##Làm sạch dữ liệu
Chuyển đổi đồ thị sang vô hướng (Undirected)

Loại bỏ các cạnh trùng lặp (Duplicate edges)

Loại bỏ các vòng tự kết nối


In [3]:
def clean_graph(G):
    # 1. Chuyển sang đồ thị vô hướng nếu cần
    if G.is_directed():
        G = G.to_undirected()

    # 2. Loại self-loops
    self_loops = list(nx.selfloop_edges(G))
    G.remove_edges_from(self_loops)

    # 3. Loại các cạnh trùng lặp bằng cách tạo Graph mới
    unique_edges = {(min(u, v), max(u, v)) for u, v in G.edges()}
    G_simple = nx.Graph()
    G_simple.add_edges_from(unique_edges)

    return G_simple


In [4]:
G_clean_step1 = clean_graph(G_raw)

print("=== SAU LÀM SẠCH ===")
print(f"Số nút:  {G_clean_step1.number_of_nodes()}")
print(f"Số cạnh: {G_clean_step1.number_of_edges()}")


=== SAU LÀM SẠCH ===
Số nút:  4039
Số cạnh: 88234


##Trích xuất GCC
Mạng xã hội thường có các nhóm nhỏ lẻ bị tách biệt. Để mô phỏng lan truyền không bị ngắt quãng, chỉ giữ lại Thành phần Liên thông Lớn nhất (Giant Connected Component - GCC).

In [5]:
# Lấy tất cả connected components
components = list(nx.connected_components(G_clean_step1))

# Lấy component lớn nhất
GCC_nodes = max(components, key=len)

# Tạo graph GCC
G_clean = G_clean_step1.subgraph(GCC_nodes).copy()

print("=== GIANT CONNECTED COMPONENT (GCC) ===")
print(f"Số nút (GCC):  {G_clean.number_of_nodes()}")
print(f"Số cạnh (GCC): {G_clean.number_of_edges()}")


=== GIANT CONNECTED COMPONENT (GCC) ===
Số nút (GCC):  4039
Số cạnh (GCC): 88234


##Output

In [7]:
OUTPUT_FILE = "facebook_combined_cleaned.txt"
nx.write_edgelist(G_clean, OUTPUT_FILE, data=False)


